In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import scipy

import gm2
import trfp
import plotting_functions as plt2
import analysis_helper as helper
import helper_function_candidates as helper_old

import mu_avg.mu_avg as mu_avg

In [ ]:
def probe_uncertainties(interp_df):
    interp_df = interp_df[['tr'+str(i) for i in range(17)]]
    probe_uncs = []
    for probe in range(17):
        diffs = np.diff(interp_df['tr'+str(probe)])
        diffs = diffs - np.polyval(np.polyfit(range(len(diffs)), diffs, 2), range(len(diffs)))
        probe_uncs.append(np.std(diffs))
    probe_uncs = np.array(probe_uncs)
    
    return probe_uncs

def multipole(order, skew, strength, x_pos, y_pos):
    """Returns the magnitude of a B-field given multipole parameters
    and position.
    """
    # Takes a multipole strength A normalized to 4.5 cm
    r_pos = np.sqrt(x_pos**2 + y_pos**2)
    theta = np.arctan2(y_pos, x_pos)
    if skew == 0:
        b_magnitude = strength * (r_pos/4.5)**order * np.cos(order*theta)
    if skew == 1:
        b_magnitude = strength * (r_pos/4.5)**order * np.sin(order*theta)
    return b_magnitude

all_ns =    [0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 7, 7, 8, 10, 12]
all_skews = [0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0]

def calc_chi2s_single(interp_df, probe_uncs, pos=0, order=1):
    
    interp_df = interp_df[['tr'+str(tr) for tr in range(17)]]
    meas = interp_df.iloc[pos].values
    chi2s = []
    
    instance_ns = [all_ns[0], all_ns[order]]
    instance_skews = [all_skews[0], all_skews[order]]
    
    for fit_order in range(1,3):
        ns = instance_ns[0:fit_order]
        skews = instance_skews[0:fit_order]

        mults = np.array([multipole(ns[i], skews[i], 1, trfp.TR_X, trfp.TR_Y) for i in range(fit_order)])
        mults[np.abs(mults) < 1.0e-9] = 0

        theta_tr = np.linalg.pinv(np.transpose(mults))
        theta_tr[np.abs(theta_tr) < 1.0e-9] = 0

        # run a set of trolley probe measurements through the matrix
        
        fit_coeffs = np.matmul(theta_tr, meas)

        fit_values = np.zeros(17)
        for i in range(fit_order):
            fit_values += multipole(ns[i], skews[i], fit_coeffs[i], trfp.TR_X, trfp.TR_Y)

        resids = meas - fit_values

        probe_uncs = np.full(17, 3)

        chi2 = 0
        for i in range(17):
            chi2 += resids[i]**2/probe_uncs[i]**2

        chi2s.append(chi2)
    
    chi2s = np.array(chi2s)
    dof = np.array([17 - m for m in range(1,3)])
    dof_Nm1 = np.array([17 - m - 1 for m in range(1,2)])
    F_chi = (chi2s[:-1] - chi2s[1:])/(chi2s[1:]/dof_Nm1)
    
    return chi2s, dof, F_chi[0]

def calc_chi2s_sequential(interp_df, probe_uncs, pos=0, through=16):
    
    interp_df = interp_df[['tr'+str(tr) for tr in range(17)]]
    meas = interp_df.iloc[pos].values
    chi2s = []

    for fit_order in range(1,through+1):

        # make a theta_tr matrix using only a set number of multipoles

        ns = all_ns[0:fit_order]
        skews = all_skews[0:fit_order]

        mults = np.array([multipole(ns[i], skews[i], 1, trfp.TR_X, trfp.TR_Y) for i in range(fit_order)])
        mults[np.abs(mults) < 1.0e-9] = 0

        theta_tr = np.linalg.pinv(np.transpose(mults))
        theta_tr[np.abs(theta_tr) < 1.0e-9] = 0

        # run a set of trolley probe measurements through the matrix
        
        fit_coeffs = np.matmul(theta_tr, meas)

        fit_values = np.zeros(17)
        for i in range(fit_order):
            fit_values += multipole(ns[i], skews[i], fit_coeffs[i], trfp.TR_X, trfp.TR_Y)

        resids = meas - fit_values

        probe_uncs = np.full(17, 3)

        chi2 = 0
        for i in range(17):
            chi2 += resids[i]**2/probe_uncs[i]**2

        chi2s.append(chi2)
    
    chi2s = np.array(chi2s)
    dof = np.array([17 - m for m in range(1,through+1)])
    dof_Nm1 = np.array([17 - m - 1 for m in range(1,through)])
    F_chi = (chi2s[:-1] - chi2s[1:])/(chi2s[1:]/dof_Nm1)
    
    return chi2s, dof, F_chi

print 'Functions defined.'

Read in a stationary trolley run. Use the full run to estimate the uncertainty in the probe measurements by looking at the differences between subsequent measurements.

In [ ]:
mults = np.array([multipole(all_ns[i], all_skews[i], 1, trfp.TR_X, trfp.TR_Y) for i in range(len(all_ns))])
mults[np.abs(mults) < 1.0e-9] = 0

theta_tr_all = np.linalg.pinv(np.transpose(mults))
theta_tr_all[np.abs(theta_tr_all) < 1.0e-9] = 0

test_m = np.matmul(theta_tr_all, np.transpose(mults))
test_m[np.abs(test_m) < 1.0e-4] = 0

# print 'If the multipole functions are properly defined, the following should be the 17x17 identity matrix.\n'
# print np.round(test_m,2)

In [ ]:
interp_df = pd.read_hdf('hdf5/calibration_region.h5', 'tr_df')
probe_uncs = probe_uncertainties(interp_df)

In [ ]:
print probe_uncs
print '\n'
print np.matmul(theta_tr_all, interp_df[['tr'+str(tr) for tr in range(17)]].iloc[500].values)

In [ ]:
skew_labels = ['norm', 'skew']

print "After fitting just a dipole, incorporating a single other moment in the fit yields:\n"
for order in range(1,17):
    _, _, F_chi = calc_chi2s_single(interp_df, probe_uncs, pos=500, order=order)
    print skew_labels[all_skews[order]] + ' ' + str(2*all_ns[order]+2) + '-pole: F = ' + str(F_chi)

In [ ]:
chi2s, dof, F_chi = calc_chi2s_sequential(interp_df, probe_uncs, pos=500)

print chi2s
print chi2s/dof
print F_chi

plt.plot(F_chi, '.')